In [1]:
import MyDL


In [2]:

a = MyDL.MyTensor([[1,2,3], [4,5,6]])
b = MyDL.MyTensor([1,2,3])
c = a + b
c.backward()
a.grad

ah


array([[1, 1, 1],
       [1, 1, 1]])

In [9]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

def sliding_window_view(img, kernel_size):
    h, w = img.shape
    k_h, k_w = kernel_size

    # 输出形状：每个窗口的左上角为一项，共(h-k+1, w-k+1)个窗口，每个窗口大小为(k_h, k_w)
    shape = (h - k_h + 1, w - k_w + 1, k_h, k_w)

    # 构造新的 stride：移动到下一个滑窗 = 向右1元素(8字节)，向下1行(8×w)
    strides = img.strides
    new_strides = (strides[0], strides[1], strides[0], strides[1])

    return as_strided(img, shape=shape, strides=new_strides)

img = np.arange(25).reshape(5, 5)
ker = [[1,1,1],[1,1,1],[1,1,1]]
windows = sliding_window_view(img, (3, 3))

print(windows.shape)  # (3, 3, 3, 3)
print(img)
print(np.sum(np.sum(windows * ker, axis=3), axis=2))


(3, 3, 3, 3)
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[ 54  63  72]
 [ 99 108 117]
 [144 153 162]]


In [ ]:
# 单通道卷积
import numpy as np
from numpy.lib.stride_tricks import as_strided

def conv2d_strided(X, K, padding=0, stride=1):
    """
    用 as_strided 实现高效的二维卷积（单通道）
    参数：
        X : 原始图像 (H, W)
        K : 卷积核 (kH, kW)
        padding : 填充边界（默认0）
        stride : 步幅（默认1）
    返回：
        Y : 卷积结果
    """
    X = np.array(X)
    K = np.array(K)

    H, W = X.shape
    kH, kW = K.shape

    # 添加 zero-padding
    if padding > 0:
        X_padded = np.pad(X, ((padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape

    # 输出尺寸
    out_h = (H_p - kH) // stride + 1
    out_w = (W_p - kW) // stride + 1

    # 构造滑动窗口视图：(out_h, out_w, kH, kW)
    shape = (out_h, out_w, kH, kW)
    strides = (
        X_padded.strides[0] * stride,
        X_padded.strides[1] * stride,
        X_padded.strides[0],
        X_padded.strides[1],
    )
    X_strided = as_strided(X_padded, shape=shape, strides=strides)

    # 逐窗口点乘卷积核并求和（向量化）
    Y = np.einsum('ijkl,kl->ij', X_strided, K)

    return Y

X = np.array([
    [1, 2, 3, 0],
    [0, 1, 2, 1],
    [1, 0, 2, 1],
    [2, 1, 3, 0]
])

K = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

Y = conv2d_strided(X, K, padding=1, stride=1)
print(Y)


[[-3 -4  2  5]
 [-3 -5  1  7]
 [-2 -4  0  7]
 [-1 -2  0  5]]


In [13]:
# 多通道卷积
import numpy as np
from numpy.lib.stride_tricks import as_strided

def conv2d_multi_channel(X, K, padding=0, stride=1):
    """
    多通道二维卷积实现（无for循环）
    参数：
        X: 输入图像 (C_in, H, W)
        K: 卷积核 (C_out, C_in, kH, kW)
        padding: 零填充大小
        stride: 步幅
    返回：
        Y: 输出特征图 (C_out, H_out, W_out)
    """
    C_in, H, W = X.shape
    C_out, C_in_k, kH, kW = K.shape
    assert C_in == C_in_k, "卷积核的输入通道数必须等于输入图像的通道数"

    # padding 每个通道
    if padding > 0:
        X_padded = np.pad(X, ((0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape[1], X_padded.shape[2]

    H_out = (H_p - kH) // stride + 1
    W_out = (W_p - kW) // stride + 1

    # 滑动窗口展开：输出形状 (C_in, H_out, W_out, kH, kW)
    strides = X_padded.strides
    new_shape = (C_in, H_out, W_out, kH, kW)
    new_strides = (
        strides[0],
        strides[1] * stride,
        strides[2] * stride,
        strides[1],
        strides[2]
    )
    X_strided = as_strided(X_padded, shape=new_shape, strides=new_strides)

    # 卷积操作：K 是 (C_out, C_in, kH, kW)，X 是 (C_in, H_out, W_out, kH, kW)
    # 使用einsum进行高效批量计算
    Y = np.einsum('oimn,ihwmn->ohw', K, X_strided)

    return Y

X = np.random.rand(3, 5, 5)   # 输入通道 C_in = 3
K = np.random.rand(4, 3, 3, 3) # 输出通道 C_out = 4，卷积核 3x3

Y = conv2d_multi_channel(X, K, padding=1, stride=1)

print(Y.shape)  # 应输出 (4, 5, 5)


(4, 5, 5)


In [ ]:
# 多通道卷积
import numpy as np
from numpy.lib.stride_tricks import as_strided

def conv2d_multi_channel(X, K, padding=0, stride=1):
    """
    多通道二维卷积实现（无for循环）
    参数：
        X: 输入图像 (batch, C_in, H, W)
        K: 卷积核 (C_out, C_in, kH, kW)
        padding: 零填充大小
        stride: 步幅
    返回：
        Y: 输出特征图 (C_out, H_out, W_out)
    """
    batch, C_in, H, W = X.shape
    C_out, C_in_k, kH, kW = K.shape
    assert C_in == C_in_k, "卷积核的输入通道数必须等于输入图像的通道数"

    # padding 每个通道
    if padding > 0:
        X_padded = np.pad(X, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape[2], X_padded.shape[3]

    H_out = (H_p - kH) // stride + 1
    W_out = (W_p - kW) // stride + 1

    # 滑动窗口展开：输出形状 (C_in, H_out, W_out, kH, kW)
    strides = X_padded.strides
    new_shape = (batch, C_in, H_out, W_out, kH, kW)
    new_strides = (
        strides[0],
        strides[1],
        strides[2] * stride,
        strides[3] * stride,
        strides[2],
        strides[3]
    )
    X_strided = as_strided(X_padded, shape=new_shape, strides=new_strides)

    # 卷积操作：K 是 (C_out, C_in, kH, kW)，X 是 (batch, C_in, H_out, W_out, kH, kW)
    # 使用einsum进行高效批量计算
    Y = np.einsum('oimn,bihwmn->bohw', K, X_strided)

    return Y


X = np.random.rand(8, 3, 5, 5)   # 输入通道 C_in = 3
K = np.random.rand(4, 3, 3, 3) # 输出通道 C_out = 4，卷积核 3x3

Y = conv2d_multi_channel(X, K, padding=1, stride=1)

print(Y.shape)  # 应输出 (8, 4, 5, 5)

(8, 4, 5, 5)
